In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
import requests
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import math
import statistics
import sys

In [2]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,1,Aaron Jones,RB,GB,27,Matt,BCDE
1,2,Jonathan Taylor,RB,IND,64,Matt,E
2,3,Odell Beckham Jr.,WR,CLE,5,Matt,E
3,4,Jerry Jeudy,WR,DEN,5,Matt,E
4,5,Davante Adams,WR,GB,60,Dai,ABCDE


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

In [4]:
transaction_list = []
driver = webdriver.Chrome('C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\chromedriver.exe')

for n in np.arange(0, 40):
    try:
        p = str(n*25)
        driver.get("https://football.fantasysports.yahoo.com/f1/597209/transactions?transactionsfilter=all&count=" + p)
        time.sleep(2)
        transaction_html = driver.page_source
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("table", {"class": "Table Table-mid Tst-transaction-table"})[0].findAll("tbody")[0]
        for tr in transaction_table.findAll('tr'):
            td = tr.findAll('td')
            if str(td[0]).find("Added Player") != -1 and str(td[0]).find("Dropped Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                player_dropped = td[1].find_all("a", {'target': 'sports'})[1].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0    
                transaction_dict = {
                    'transaction_type': 'drop-add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Dropped Player") != -1:
                player_dropped = td[1].find_all("a", {'target': 'sports'})[0].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_dict = {
                    'transaction_type': 'drop',
                    'player_added': 'Bench',
                    'player_added_team': 'N/A',
                    'player_added_pos': 'N/A',
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': 0
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Added Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0 
                transaction_dict = {
                    'transaction_type': 'add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': 'Bench',
                    'player_dropped_team': "N/A",
                    'player_dropped_pos': "N/A",
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)

            elif str(td[1]).find("Traded to") != -1:
                manager = td[2].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[0].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[0].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
            elif str(td[2]).find("Traded to") != -1:
                manager = td[3].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[3].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[1].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[1].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
    except:
        pass

driver.quit()
transaction_df = pd.DataFrame(transaction_list)
transaction_df.head()

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type
0,Amon The Stars,D'Ernest Johnson,RB,Cle,San Francisco,DEF,SF,0,"Jan 2, 4:34 pm",drop-add
1,Zeke and Ye Shall Find,Deonte Harris,WR,NO,Emmanuel Sanders,WR,Buf,0,"Jan 2, 9:24 am",drop-add
2,Zeke and Ye Shall Find,Trey Sermon,RB,SF,Philadelphia,DEF,Phi,0,"Jan 2, 9:08 am",drop-add
3,Zeke and Ye Shall Find,Gabriel Davis,WR,Buf,Tyler Johnson,WR,TB,0,"Dec 31, 3:46 pm",drop-add
4,folklore,Josh Gordon,WR,KC,Joshua Palmer,WR,LAC,0,"Dec 31, 1:02 pm",drop-add


In [5]:
manager_list = []
for m in list(set(transaction_df['ff_team'])):
    manager = input("Which player is the manager for " + m + "?")
    manager_dict = {
        'ff_team': m,
        'manager': manager
    }
    manager_list.append(manager_dict)
manager_df = pd.DataFrame(manager_list)
manager_df
    

Which player is the manager for Raider Bae?Jake
Which player is the manager for Waddle It Be??Sean
Which player is the manager for Zeke and Ye Shall Find?Doug
Which player is the manager for Got Taylor Need Swift?Matt
Which player is the manager for Chi ShingT's Team?Chi Shing
Which player is the manager for Amon The Stars?Joel
Which player is the manager for Tiz the Law?Dai
Which player is the manager for folklore?Ron
Which player is the manager for Pop Drop and Lockett?Evan
Which player is the manager for Nags?Ryan
Which player is the manager for G?Jiwei
Which player is the manager for Sacko?Rajiv


,ff_team,manager
0,Raider Bae,Jake
1,Waddle It Be?,Sean
2,Zeke and Ye Shall Find,Doug
3,Got Taylor Need Swift,Matt
4,Chi ShingT's Team,Chi Shing
5,Amon The Stars,Joel
6,Tiz the Law,Dai
7,folklore,Ron
8,Pop Drop and Lockett,Evan
9,Nags,Ryan


In [7]:
transaction_df = pd.DataFrame(transaction_list)
for i, r in transaction_df.iterrows():
    if r['transaction_time'].find('Jan') != -1:
        transaction_df.at[i, 'transaction_time'] = "2022 " + r['transaction_time']
    else:
        transaction_df.at[i, 'transaction_time'] = "2021 " + r['transaction_time']
transaction_df['transaction_time'] = pd.to_datetime(transaction_df['transaction_time'], format="%Y %b %d, %I:%M %p")
transaction_df = transaction_df.sort_values(by='transaction_time').reset_index(drop=True)
transaction_df = transaction_df.merge(manager_df, on='ff_team', how='left')
transaction_df.to_csv("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Transactions.csv")
transaction_df.head()

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type,manager
0,Pop Drop and Lockett,Ty'Son Williams,RB,Bal,Los Angeles,DEF,LAC,7,2021-08-30 00:09:00,drop-add,Evan
1,Amon The Stars,Jermar Jefferson,RB,Det,Kirk Cousins,QB,Min,5,2021-08-30 00:09:00,drop-add,Joel
2,Got Taylor Need Swift,Nelson Agholor,WR,NE,Todd Gurley II,RB,Atl,0,2021-08-30 00:09:00,drop-add,Matt
3,Waddle It Be?,Sterling Shepard,WR,NYG,Russell Gage,WR,Atl,0,2021-08-30 00:10:00,drop-add,Sean
4,Waddle It Be?,Tony Jones Jr.,RB,NO,Latavius Murray,RB,Bal,0,2021-08-30 00:13:00,drop-add,Sean


In [12]:
transaction_df[24:25]

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type,manager
24,Raider Bae,New England,DEF,NE,Phillip Lindsay,RB,Mia,0,2021-09-05 12:19:00,drop-add,Jake


In [14]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Draft.xlsx")
draft_df

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,1,Aaron Jones,RB,GB,27,Matt,BCDE
1,2,Jonathan Taylor,RB,IND,64,Matt,E
2,3,Odell Beckham Jr.,WR,CLE,5,Matt,E
3,4,Jerry Jeudy,WR,DEN,5,Matt,E
4,5,Davante Adams,WR,GB,60,Dai,ABCDE
5,6,Miles Sanders,RB,Phi,41,Dai,DE
6,7,DJ Moore,WR,Car,26,Dai,CDE
7,8,Zack Moss,RB,Buf,16,Dai,E
8,9,Joe Mixon,RB,CIN,72,Chi Shing,CDE
9,10,CeeDee Lamb,WR,DAL,23,Chi Shing,E


In [34]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

for i, r in transaction_df.iterrows():
    print(i, r['transaction_type'], r['player_added'], r['player_added_team'], r['player_dropped'], r['player_dropped_team'])
    if r['transaction_type'] == 'drop-add':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped'])]
        draft_df.at[draft_row_n.index[0], 'player_name'] = r['player_added']
        draft_df.at[draft_row_n.index[0], 'player_team'] = r['player_added_team']
        draft_df.at[draft_row_n.index[0], 'player_position'] = r['player_added_pos']
        draft_df.at[draft_row_n.index[0], 'salary'] = r['transaction_cost']
        draft_df.at[draft_row_n.index[0], 'keeper_code'] = ''
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'drop':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped'])]
        draft_df = draft_df.drop([draft_row_n.index[0]]).reset_index(drop=True)
    elif r['transaction_type'] == 'add':
        draft_df = draft_df.append({'pick': 0, 
                         'player_name': r['player_added'], 
                         'player_position': r['player_added_pos'], 
                         'player_team': r['player_added_team'],
                         'salary': r['transaction_cost'],
                         'manager': r['manager'],
                         'keeper_code': ''
                        }, ignore_index=True)
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'trade':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_added'])]
        draft_df.at[draft_row_n.index[0], 'manager'] = r['manager']
        draft_df.reset_index(drop=True)
draft_df = draft_df.sort_values(by=['manager', 'salary']).reset_index(drop=True)

0 drop-add Ty'Son Williams Bal Los Angeles LAC
1 drop-add Jermar Jefferson Det Kirk Cousins Min
2 drop-add Nelson Agholor NE Todd Gurley II Atl
3 drop-add Sterling Shepard NYG Russell Gage Atl
4 drop-add Tony Jones Jr. NO Latavius Murray Bal
5 drop-add Ty Johnson NYJ Leonard Fournette TB
6 drop-add Tyrell Williams Det Allen Lazard GB
7 drop-add Aldrick Rosas Det Wil Lutz NO
8 drop-add Tua Tagovailoa Mia Cole Kmet Chi
9 drop-add Rodrigo Blankenship Ind Aldrick Rosas Det
10 drop-add Jalen Reagor Phi John Brown Jax
11 drop-add Devontae Booker NYG New Orleans NO
12 drop-add Chris Evans Cin Sam Darnold Car
13 drop-add Miami Mia New England NE
14 drop-add Kenneth Gainwell Phi Justice Hill Bal
15 drop-add Daniel Carlson LV Nick Folk NE
16 drop-add Todd Gurley II Atl Juwan Johnson NO
17 drop-add Jason Myers Sea Brandon McManus Den
18 drop-add Bryan Edwards LV J.K. Dobbins Bal
19 drop-add Rondale Moore Ari Tony Jones Jr. NO
20 drop-add Mac Jones NE Deshaun Watson Hou
21 drop-add Wayne Gallman J

311 drop-add Rondale Moore Ari Rex Burkhead Hou
312 drop-add Brandon Aiyuk SF DeVante Parker Mia
313 drop-add Jimmy Garoppolo SF Sammy Watkins Bal
314 drop-add James Washington Pit Logan Thomas Was
315 drop-add Cole Kmet Chi Michael Thomas NO
316 drop-add Tennessee Ten New Orleans NO
317 drop-add Devonta Freeman Bal Peyton Barber LV
318 drop-add Arizona Ari Ty'Son Williams Bal
319 drop-add Baltimore Bal Cincinnati Cin
320 drop-add Logan Thomas Was Greg Zuerlein Dal
321 drop-add Malik Turner Dal K.J. Osborn Min
322 drop-add T.Y. Hilton Ind Latavius Murray Bal
323 drop-add Michael Thomas NO Foster Moreau LV
324 drop-add DeSean Jackson LV Bryan Edwards LV
325 drop-add Chris Boswell Pit Jeff Wilson Jr. SF
326 drop-add Denver Den Chicago Chi
327 drop-add Younghoe Koo Atl Brandon McManus Den
328 drop-add Deonte Harris NO James Washington Pit
329 drop-add Eno Benjamin Ari Cole Kmet Chi
330 drop-add Rashaad Penny Sea Dallas Dal
331 drop-add Philadelphia Phi San Francisco SF
332 drop-add Rhamon

In [35]:
draft_df.to_csv('C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\yahoo_keeper_2021.csv')

In [36]:
draft_df[draft_df['manager'] == 'Chi Shing']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,96,Darrell Henderson Jr.,RB,LAR,0,Chi Shing,
1,132,David Njoku,TE,Cle,0,Chi Shing,
2,145,Matt Gay,K,LAR,0,Chi Shing,
3,157,Marlon Mack,RB,Ind,0,Chi Shing,
4,129,Christian Kirk,WR,Ari,8,Chi Shing,
5,126,DeVonta Smith,WR,Phi,12,Chi Shing,NaN
6,71,Melvin Gordon III,RB,Den,21,Chi Shing,NaN
7,10,CeeDee Lamb,WR,DAL,23,Chi Shing,E
8,11,Patrick Mahomes,QB,KC,24,Chi Shing,CDE
9,91,Tyler Higbee,TE,LAR,25,Chi Shing,NaN


In [37]:
draft_df[draft_df['manager'] == 'Dai']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
16,8,DeVante Parker,WR,Mia,0,Dai,
17,119,Khalil Herbert,RB,Chi,0,Dai,
18,127,Ke'Shawn Vaughn,RB,TB,0,Dai,
19,138,Kadarius Toney,WR,NYG,0,Dai,
20,139,Rashaad Penny,RB,Sea,0,Dai,
21,169,Rashod Bateman,WR,Bal,0,Dai,
22,170,Derrick Gore,RB,KC,0,Dai,
23,182,JuJu Smith-Schuster,WR,Pit,0,Dai,
24,162,Michael Carter,RB,NYJ,3,Dai,NaN
25,40,Josh Allen,QB,Buf,23,Dai,E


In [38]:
draft_df[draft_df['manager'] == 'Doug']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
32,57,Jaret Patterson,RB,Was,0,Doug,
33,97,Allen Lazard,WR,GB,0,Doug,
34,147,Jeff Wilson Jr.,RB,SF,0,Doug,
35,148,Buffalo,DEF,Buf,0,Doug,
36,185,Deonte Harris,WR,NO,0,Doug,
37,0,Rhamondre Stevenson,RB,NE,0,Doug,
38,0,Gabriel Davis,WR,Buf,0,Doug,
39,0,Trey Sermon,RB,SF,0,Doug,
40,168,Jakobi Meyers,WR,NE,1,Doug,NaN
41,27,Justin Herbert,QB,SD,5,Doug,E


In [39]:
draft_df[draft_df['manager'] == 'Evan']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
48,73,Darrel Williams,RB,KC,0,Evan,
49,121,Ben Roethlisberger,QB,Pit,0,Evan,
50,133,Jared Cook,TE,LAC,0,Evan,
51,140,Ryan Succop,K,TB,0,Evan,
52,163,Denver,DEF,Den,0,Evan,
53,152,Matt Prater,K,Ari,2,Evan,
54,153,Darnell Mooney,WR,Chi,4,Evan,NaN
55,116,Michael Gallup,WR,Dal,11,Evan,NaN
56,128,Jarvis Landry,WR,Cle,15,Evan,
57,85,Tyler Boyd,WR,Cin,18,Evan,NaN


In [40]:
draft_df[draft_df['manager'] == 'Jake']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
64,25,Gerald Everett,TE,Sea,0,Jake,
65,83,DJ Chark Jr.,WR,Jax,0,Jake,
66,84,Miami,DEF,Mia,0,Jake,
67,101,Marquez Callaway,WR,NO,0,Jake,
68,146,Boston Scott,RB,Phi,0,Jake,
69,151,Mac Jones,QB,NE,0,Jake,
70,156,A.J. Green,WR,Ari,0,Jake,
71,106,Justin Tucker,K,Bal,2,Jake,NaN
72,112,Cole Beasley,WR,Buf,2,Jake,NaN
73,74,James Conner,RB,Ari,16,Jake,NaN


In [41]:
draft_df[draft_df['manager'] == 'Jiwei']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
80,28,Alex Collins,RB,Sea,0,Jiwei,
81,100,Chris Boswell,K,Pit,0,Jiwei,
82,131,Kendrick Bourne,WR,NE,0,Jiwei,
83,137,New Orleans,DEF,NO,0,Jiwei,
84,149,Kenny Golladay,WR,NYG,0,Jiwei,
85,160,Ameer Abdullah,RB,Car,0,Jiwei,
86,72,Jalen Hurts,QB,Phi,7,Jiwei,NaN
87,117,Marvin Jones Jr.,WR,Jax,7,Jiwei,NaN
88,45,Justin Jefferson,WR,Min,9,Jiwei,E
89,141,Alexander Mattison,RB,Min,11,Jiwei,


In [42]:
draft_df[draft_df['manager'] == 'Joel']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
96,32,Pat Freiermuth,TE,Pit,0,Joel,
97,33,Amon-Ra St. Brown,WR,Det,0,Joel,
98,51,D'Ernest Johnson,RB,Cle,0,Joel,
99,61,Foster Moreau,TE,LV,0,Joel,
100,165,Leonard Fournette,RB,TB,0,Joel,
101,177,Tyler Bass,K,Buf,0,Joel,
102,93,Brandin Cooks,WR,Hou,11,Joel,NaN
103,104,Deebo Samuel,WR,SF,13,Joel,NaN
104,34,Aaron Rodgers,QB,GB,15,Joel,E
105,81,Tee Higgins,WR,Cin,28,Joel,NaN


In [43]:
draft_df[draft_df['manager'] == 'Matt']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
112,4,Zack Moss,RB,Buf,0,Matt,
113,12,Mike Gesicki,TE,Mia,0,Matt,
114,80,Raheem Mostert,RB,SF,0,Matt,
115,99,Tim Patrick,WR,Den,0,Matt,
116,102,Dalton Schultz,TE,Dal,0,Matt,
117,108,Cincinnati,DEF,Cin,0,Matt,
118,109,Travis Etienne,RB,Jax,0,Matt,
119,120,Jordan Howard,RB,Phi,0,Matt,
120,150,Michael Badgley,K,Ind,0,Matt,
121,190,Brandon Aiyuk,WR,SF,0,Matt,


In [44]:
draft_df[draft_df['manager'] == 'Rajiv']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
128,95,Matt Ryan,QB,Atl,0,Rajiv,
129,113,Los Angeles,DEF,LAC,0,Rajiv,
130,143,Nyheim Hines,RB,Ind,0,Rajiv,
131,166,Younghoe Koo,K,Atl,0,Rajiv,
132,187,Hunter Henry,TE,NE,0,Rajiv,
133,0,Noah Fant,TE,Den,0,Rajiv,
134,0,Tre'Quan Smith,WR,NO,0,Rajiv,
135,179,Devin Singletary,RB,Buf,1,Rajiv,NaN
136,171,Marquise Brown,WR,Bal,2,Rajiv,NaN
137,184,Hunter Renfrow,WR,LV,6,Rajiv,


In [45]:
draft_df[draft_df['manager'] == 'Ron']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
144,20,Green Bay,DEF,GB,0,Ron,
145,53,Trey Lance,QB,SF,0,Ron,
146,60,Josh Gordon,WR,KC,0,Ron,
147,110,K.J. Osborn,WR,Min,0,Ron,
148,123,Joe Burrow,QB,Cin,0,Ron,
149,135,Cordarrelle Patterson,"WR,RB",Atl,0,Ron,
150,158,Chicago,DEF,Chi,0,Ron,
151,172,Dawson Knox,TE,Buf,4,Ron,
152,17,DK Metcalf,WR,SEA,12,Ron,DE
153,111,Dallas Goedert,TE,Phi,13,Ron,NaN


In [46]:
draft_df[draft_df['manager'] == 'Ryan']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
160,114,Daniel Carlson,K,LV,0,Ryan,
161,173,J.K. Dobbins,RB,Bal,0,Ryan,
162,175,Rob Gronkowski,TE,TB,1,Ryan,NaN
163,174,Tony Pollard,RB,Dal,2,Ryan,NaN
164,176,Jamaal Williams,RB,Det,2,Ryan,NaN
165,124,Tampa Bay,DEF,TB,4,Ryan,NaN
166,37,Chase Claypool,WR,PIT,5,Ryan,E
167,118,Laviska Shenault Jr.,WR,Jax,10,Ryan,NaN
168,88,Tom Brady,QB,TB,13,Ryan,NaN
169,36,Nick Chubb,RB,CLE,24,Ryan,CDE


In [47]:
draft_df[draft_df['manager'] == 'Sean']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
176,142,Seattle,DEF,Sea,0,Sean,
177,164,Elijah Moore,WR,NYJ,0,Sean,
178,167,Zach Ertz,TE,Ari,0,Sean,
179,178,Evan McPherson,K,Cin,0,Sean,
180,188,T.J. Hockenson,TE,Det,0,Sean,
181,189,Cam Akers,RB,LAR,0,Sean,
182,16,Dak Prescott,QB,DAL,5,Sean,E
183,154,Jaylen Waddle,WR,Mia,5,Sean,NaN
184,90,Mike Williams,WR,LAC,13,Sean,NaN
185,14,AJ Brown,WR,TEN,18,Sean,DE
